실습 코드는 PyTorch Tutorials의 “[NN.TRANSFORMER와 TORCHTEXT로 언어 번역하기](https://tutorials.pytorch.kr/beginner/translation_transformer.html)” 문서를 참고했음을 밝힙니다.

# 사전 준비

- [spacy tokenizer](https://wikidocs.net/64517)

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     |████████████████████████████████| 6.0 MB 14.8 MB/s 
     |████████████████████████████████| 628 kB 71.0 MB/s 
     |████████████████████████████████| 451 kB 60.7 MB/s 
     |████████████████████████████████| 10.1 MB 42.7 MB/s 
     |████████████████████████████████| 181 kB 73.6 MB/s 
     |████████████████████████████████| 42 kB 1.9 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 13.9 

# 데이터셋

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import time
import random

In [ ]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.datasets import Multi30k
from typing import Iterable, List

In [ ]:
# 사용할 데이터셋 확인
## 학습용 데이터 반복자
SRC_LANGUAGE = 'de'
TGT_LANGUAGE = 'en'

train_iter = Multi30k(root='./data', split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

for idx, data_sample in enumerate(train_iter):
  if idx > 10: break
  print(idx, data_sample)

100%|██████████| 1.21M/1.21M [00:00<00:00, 4.98MB/s]

0 ('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.\n', 'Two young, White males are outside near many bushes.\n')
1 ('Mehrere Männer mit Schutzhelmen bedienen ein Antriebsradsystem.\n', 'Several men in hard hats are operating a giant pulley system.\n')
2 ('Ein kleines Mädchen klettert in ein Spielhaus aus Holz.\n', 'A little girl climbing into a wooden playhouse.\n')
3 ('Ein Mann in einem blauen Hemd steht auf einer Leiter und putzt ein Fenster.\n', 'A man in a blue shirt is standing on a ladder cleaning a window.\n')
4 ('Zwei Männer stehen am Herd und bereiten Essen zu.\n', 'Two men are at the stove preparing food.\n')
5 ('Ein Mann in grün hält eine Gitarre, während der andere Mann sein Hemd ansieht.\n', 'A man in green holds a guitar while the other man observes his shirt.\n')
6 ('Ein Mann lächelt einen ausgestopften Löwen an.\n', 'A man is smiling at a stuffed lion\n')
7 ('Ein schickes Mädchen spricht mit dem Handy während sie langsam die Straße entlangschwebt.\n',

In [ ]:
# tokenizer
token_transform = {}
token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_sm')
token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_sm')

print(token_transform[SRC_LANGUAGE]("Eine Gruppe von Menschen steht vor einem Iglu ."))

['Eine', 'Gruppe', 'von', 'Menschen', 'steht', 'vor', 'einem', 'Iglu', '.']


In [ ]:
# 특수 토큰
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

UNK_TKN_IDX = 0
PAD_TKN_IDX = 1
SOS_TKN_IDX = 2
EOS_TKN_IDX = 3

`build_vocab_from_iterator` 함수를 사용해 vocab set을 생성할 수 있다. [pytorch docs](https://pytorch.org/text/stable/vocab.html#build-vocab-from-iterator)

이때 생성되는 vocab set은 `toechtext.vocab.Vocab` 객체인데, `get_itos()`, `get_stoi()` 등의 함수로 vocab set의 기능을 제공한다. [pytorch docs](https://pytorch.org/text/stable/vocab.html#vocab)

In [ ]:
# vocabulary set(어휘집) 생성
vocab_transform = {} # integer encoding을 수행하는 vocab set의 묶음

# (helper function) 데이터셋에서 특정 language의 것만 반환하는 generator function.
# `yield` 키워드에 주목하자.
def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
  language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}
  tokenizer = token_transform[language]
  
  for data_sample in data_iter:
    sentence = data_sample[language_index[language]]
    yield tokenizer(sentence)

# vocab set: SRC LANGUAGE
train_iter = Multi30k(root='./data', split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
vocab_transform[SRC_LANGUAGE] \
  = build_vocab_from_iterator(
      iterator = yield_tokens(train_iter, SRC_LANGUAGE),
      min_freq = 1,
      specials = special_symbols,
      special_first = True # `specials`의 토큰이 가장 앞의 index를 가지도록 설정
    )
vocab_transform[SRC_LANGUAGE].set_default_index(UNK_TKN_IDX)

# vocab set: TGT LANGUAGE
train_iter = Multi30k(root='./data', split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
vocab_transform[TGT_LANGUAGE] \
  = build_vocab_from_iterator(
      iterator = yield_tokens(train_iter, TGT_LANGUAGE),
      min_freq = 1,
      specials = special_symbols,
      special_first = True
    )
vocab_transform[TGT_LANGUAGE].set_default_index(UNK_TKN_IDX)

print(vocab_transform[SRC_LANGUAGE].get_default_index())
print(vocab_transform[SRC_LANGUAGE](token_transform[SRC_LANGUAGE]("Eine Gruppe von Menschen steht vor einem Iglu .")))

0
[15, 39, 25, 55, 31, 29, 7, 6133, 5]


# 모델링

1. Embedding: token embedding & positional encoding
2. Transformer: `Seq2SeqTranformer`
3. Output Layer

In [ ]:
from torch import Tensor
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Embedding

## Token Embedding

In [ ]:
class TokenEmbedding(nn.Module):
  def __init__(self, vocab_size: int, emb_size: int):
    super(TokenEmbedding, self).__init__()
    self.embedding = nn.Embedding(vocab_size, emb_size)
    self.emb_size = emb_size
  
  def forward(self, tokens: Tensor):
    out = self.embedding(tokens.long())
    out *= math.sqrt(self.emb_size) # scaling
    return out

## Positional Encoding

$$
PE(pos) = 
\begin{cases}
  \sin(\omega_k \cdot pos) & \text{if} \; i = 2k \\
  \cos(\omega_k \cdot pos) & \text{if} \; i = 2k+1 
\end{cases} 
\quad \left( \omega_k = \frac{1}{1000^{k/d}} \right)
$$

<br/>

Note: $\omega_k = \frac{1}{1000^{k/d}} = \exp \left( \log \frac{1}{1000^{k/d}} \right) = \exp \left( - \log (1000^{k/d}) \right) = \exp \left( - \log (1000) \times {k/d} \right)$

In [ ]:
class PositionalEncoding(nn.Module):
  def __init__(self, emb_size:int, maxlen: int = 5000):
    super(PositionalEncoding, self).__init__()
    
    pos = torch.arange(0, maxlen).reshape(maxlen, 1)

    # $1000^{k/d}$를 실제로 계산하려고 하면 underflow가 발생할 수 있으니 log trick을 사용
    frequency = torch.exp(- math.log(1000) * (torch.arange(0, emb_size, 2) / emb_size))

    # sinusoidla encoding by sin & cos
    pos_embedding = torch.zeros((maxlen, emb_size))
    pos_embedding[:, 0::2] = torch.sin(pos * frequency) # 짝수 인덱스
    pos_embedding[:, 1::2] = torch.cos(pos * frequency) # 홀수 인덱스
    pos_embedding = pos_embedding.unsqueeze(-2)

    self.register_buffer('pos_embedding', pos_embedding) # ref. https://powerofsummary.tistory.com/158

  def forward(self, token_embedding: Tensor):
    token_length = token_embedding.size(0)
    return token_embedding + self.pos_embedding[:token_length, :]

## Seq2SeqTranformer

`nn.Transformer()` 모듈을 사용한다! [pytorch docs](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html#transformer)

In [ ]:
class Seq2SeqTransformer(nn.Module):
  def __init__(self, 
               num_encoder_layers: int, num_decoder_layers:int, 
               emb_size: int, nhead:int,
               src_vocab_size:int, tgt_vocab_size:int,  
               dim_feedforward:int = 512):
    super(Seq2SeqTransformer, self).__init__()
    
    self.src_tkn_emb = TokenEmbedding(src_vocab_size, emb_size)
    self.tgt_tkn_emb = TokenEmbedding(tgt_vocab_size, emb_size)
    self.positional_encoding = PositionalEncoding(emb_size)

    self.transformer = nn.Transformer(
        d_model = emb_size,
        num_encoder_layers = num_encoder_layers,
        num_decoder_layers = num_decoder_layers,
        dim_feedforward = dim_feedforward)

    self.output_layer = nn.Linear(emb_size, tgt_vocab_size)

  
  def forward(self, 
              src: Tensor, tgt: Tensor,
              src_mask: Tensor, tgt_mask: Tensor,
              src_pad_mask: Tensor, tgt_pad_mask: Tensor):
    # embedding
    src_emb = self.positional_encoding(self.src_tkn_emb(src))
    tgt_emb = self.positional_encoding(self.tgt_tkn_emb(tgt))

    # transformer
    outs = self.transformer(
        src_emb, tgt_emb, 
        src_mask, tgt_mask, None, 
        src_pad_mask, tgt_pad_mask, src_pad_mask
      )

    # output layer
    out = self.output_layer(outs)
    return out

  def encode(self, src: Tensor, src_mask: Tensor):
    src_emb = self.positional_encoding(self.src_tkn_emb(src))
    return self.transformer.encoder(src_emb, src_mask)

  def decode(self, tgt: Tensor, memory: Tensor, tgt_mask: Tensor):
    tgt_emb = self.positional_encoding(self.tgt_tkn_emb(tgt))
    return self.transformer.decoder(tgt_emb, memory, tgt_mask)


# 학습

## 하이퍼 파라미터

In [ ]:
torch.manual_seed(0)

SRC_VOCAB_SIZE = len(vocab_transform[SRC_LANGUAGE])
TGT_VOCAB_SIZE = len(vocab_transform[TGT_LANGUAGE])

EMB_SIZE = 512
NHEAD = 8
FFN_HID_DIM = 512
NUM_ENCODER_LAYERS = 3
NUM_DECODER_LAYERS = 3

BATCH_SIZE = 128

## 모델 선언

In [ ]:
transformer = Seq2SeqTransformer(NUM_ENCODER_LAYERS, NUM_DECODER_LAYERS, EMB_SIZE,
                                 NHEAD, SRC_VOCAB_SIZE, TGT_VOCAB_SIZE, FFN_HID_DIM)

for p in transformer.parameters():
  if p.dim() > 1:
      nn.init.xavier_uniform_(p)

transformer = transformer.to(DEVICE)

criterion = torch.nn.CrossEntropyLoss(ignore_index=PAD_TKN_IDX)
optimizer = torch.optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

## 문자열 → 배치 텐서

데이터 반복자(iterator)는 raw한 문자열의 쌍을 생성합니다. 이 문자열 쌍들을 정의한 Transformer에서 처리할 수 있도록 텐서 묶음(batched tensor)으로 변환해야 합니다. 

In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(raw_batch):
  src_batch, tgt_batch = [], []
  for src_sample, tgt_sample in raw_batch:
    src_sample = src_sample.rstrip("\n")
    src_sample = token_transform[SRC_LANGUAGE](src_sample) # tokenize
    src_sample = vocab_transform[SRC_LANGUAGE](src_sample) # integer encoding
    src_sample = torch.cat((torch.tensor([SOS_TKN_IDX]),
                      torch.tensor(src_sample),
                      torch.tensor([EOS_TKN_IDX])))
    src_batch.append(src_sample)

    tgt_sample = tgt_sample.rstrip("\n")
    tgt_sample = token_transform[TGT_LANGUAGE](tgt_sample) # tokenize
    tgt_sample = vocab_transform[TGT_LANGUAGE](tgt_sample) # integer encoding
    tgt_sample = torch.cat((torch.tensor([SOS_TKN_IDX]),
                      torch.tensor(tgt_sample),
                      torch.tensor([EOS_TKN_IDX])))
    tgt_batch.append(tgt_sample)
  
  src_batch = pad_sequence(src_batch, padding_value=PAD_TKN_IDX)
  tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_TKN_IDX)

  return src_batch, tgt_batch

train_iter = Multi30k(root='./data', split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

for idx, sample in enumerate(train_dataloader):
  if idx >= 1: break
  print(idx, sample[0].shape, sample) # (max_seq_len, batch_size)

0 torch.Size([27, 128]) (tensor([[ 2,  2,  2,  ...,  2,  2,  2],
        [22, 85,  6,  ..., 22, 15, 15],
        [86, 32, 70,  ..., 47, 39, 18],
        ...,
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1],
        [ 1,  1,  1,  ...,  1,  1,  1]]), tensor([[  2,   2,   2,  ...,   2,   2,   2],
        [ 20, 166,   7,  ...,  20,   7,   7],
        [ 26,  37,  62,  ...,  53,  40,  17],
        ...,
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1],
        [  1,   1,   1,  ...,   1,   1,   1]]))


## Mask 생성

In [ ]:
# subsequent mask도 구현
def generate_square_subsequent_mask(size: int):
  mask = torch.triu(torch.ones((size, size), device=DEVICE) == 1) # upper triangular
  mask = mask.transpose(0, 1)
  mask = mask.float().masked_fill(mask == 0, float('-inf'))
  mask = mask.float().masked_fill(mask == 1, float(0.0))
  return mask

def create_mask(src: Tensor, tgt: Tensor):
  src_seq_len = src.shape[0]
  tgt_seq_len = tgt.shape[0]

  src_pad_mask = (src == PAD_TKN_IDX).transpose(0, 1) # transpose(0, 1): make batch dim first
  tgt_pad_mask = (tgt == PAD_TKN_IDX).transpose(0, 1) # (batch_size, max_seq_len)

  src_mask = torch.zeros((src_seq_len, src_seq_len), device=DEVICE).type(torch.bool) # (max_seq_len, max_seq_len)
  tgt_mask = generate_square_subsequent_mask(tgt_seq_len)

  return src_mask, tgt_mask, src_pad_mask, tgt_pad_mask

for idx, sample in enumerate(train_dataloader):
  if idx >= 1: break
  print(idx, sample[0].shape) # (max_seq_len, batch_size)
  src_mask, tgt_mask, src_pad_mask, tgt_pad_mask = create_mask(sample[0], sample[1])
  print(src_mask.shape, tgt_mask.shape) # (max_seq_len, max_seq_len)
  print(src_pad_mask.shape, tgt_pad_mask.shape) # (batch_size, max_seq_len)

0 torch.Size([33, 128])
torch.Size([33, 33]) torch.Size([36, 36])
torch.Size([128, 33]) torch.Size([128, 36])


## Train Function

In [ ]:
# data load
def train_epoch(model, optimizer):
  model.train()
  total_loss = 0
  
  train_iter = Multi30k(root='./data', split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
  train_dataloader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

  for src, tgt in train_dataloader:
    src = src.to(DEVICE) # (max_seq_len, batch_size)
    tgt = tgt.to(DEVICE)

    tgt_input = tgt[:-1, :] # remove <eos> token?
    src_mask, tgt_mask, src_pad_mask, tgt_pad_mask = create_mask(src, tgt_input)

    logits = model(src, tgt_input, src_mask, tgt_mask, src_pad_mask, tgt_pad_mask)

    optimizer.zero_grad()

    tgt_out = tgt[1:, :]
    loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    loss.backward()

    optimizer.step()
    total_loss += loss.item()

  return total_loss / len(train_dataloader)


tic = time.time()
epoch_loss = train_epoch(transformer, optimizer)
toc = time.time()
print(f'time: {toc - tic:5.1f} sec | train loss: {epoch_loss:8.4f}')

time:  36.1 sec | train loss:   5.2571


## Eval Function

In [ ]:
def evaluate(model):
  model.eval()
  total_loss = 0

  val_iter = Multi30k(root='./data', split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
  val_dataloader = DataLoader(val_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

  for src, tgt in val_dataloader:
    src = src.to(DEVICE)
    tgt = tgt.to(DEVICE)

    tgt_input = tgt[:-1, :]
    src_mask, tgt_mask, src_pad_mask, tgt_pad_mask = create_mask(src, tgt_input)

    logits = model(src, tgt_input, src_mask, tgt_mask, src_pad_mask, tgt_pad_mask)

    tgt_out = tgt[1:, :]
    loss = criterion(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
    total_loss += loss.item()

  return total_loss / len(val_dataloader)


tic = time.time()
val_loss = evaluate(transformer)
toc = time.time()
print(f'time: {toc - tic:5.1f} sec | val loss: {val_loss:8.4f}')

100%|██████████| 46.3k/46.3k [00:00<00:00, 969kB/s]


time:   0.8 sec | val loss:   3.9802


## Epoch Train

In [ ]:
NUM_EPOCHS = 15

for epoch in range(1, NUM_EPOCHS+1):
  tic = time.time()
  train_loss = train_epoch(transformer, optimizer)
  val_loss = evaluate(transformer)
  toc = time.time()
  print(f'| epoch: {epoch:3d} | time: {toc - tic:5.1f} sec | train loss: {train_loss:6.4f} | val loss: {val_loss:6.4f}')

| epoch:   1 | time:  36.8 sec | train loss: 3.5973 | val loss: 3.1816
| epoch:   2 | time:  37.3 sec | train loss: 2.9523 | val loss: 2.7729
| epoch:   3 | time:  37.7 sec | train loss: 2.5324 | val loss: 2.5115
| epoch:   4 | time:  38.0 sec | train loss: 2.2249 | val loss: 2.3437
| epoch:   5 | time:  38.3 sec | train loss: 1.9785 | val loss: 2.2367
| epoch:   6 | time:  38.5 sec | train loss: 1.7801 | val loss: 2.1441
| epoch:   7 | time:  38.6 sec | train loss: 1.6109 | val loss: 2.0773
| epoch:   8 | time:  38.7 sec | train loss: 1.4680 | val loss: 2.0306
| epoch:   9 | time:  38.8 sec | train loss: 1.3422 | val loss: 2.0358
| epoch:  10 | time:  38.8 sec | train loss: 1.2319 | val loss: 2.0368
| epoch:  11 | time:  38.8 sec | train loss: 1.1262 | val loss: 2.0542
| epoch:  12 | time:  38.8 sec | train loss: 1.0322 | val loss: 2.0560
| epoch:  13 | time:  38.9 sec | train loss: 0.9517 | val loss: 2.0292
| epoch:  14 | time:  38.9 sec | train loss: 0.8785 | val loss: 2.0077
| epoc

# 성능 확인

In [ ]:
# 순차적인 작업들을 하나로 묶는 헬퍼 함수
def sequential_transforms(*transforms):
  def callback(txt_input):
    for transform in transforms:
      txt_input = transform(txt_input)
    return txt_input
  return callback


# BOS/EOS를 추가하고 입력 순서(sequence) 인덱스에 대한 텐서를 생성하는 함수
def tensor_transform(token_ids: List[int]):
  return torch.cat((torch.tensor([SOS_TKN_IDX]),
                    torch.tensor(token_ids),
                    torch.tensor([EOS_TKN_IDX])))

# 출발어(src)와 도착어(tgt) 원시 문자열들을 텐서 인덱스로 변환하는 변형(transform)
text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
  text_transform[ln] = sequential_transforms(token_transform[ln], # 토큰화(Tokenization)
                                               vocab_transform[ln], # 수치화(Numericalization)
                                               tensor_transform) # BOS/EOS를 추가하고 텐서를 생성

In [ ]:
def greedy_decode(model, src: Tensor, src_mask: Tensor, max_len:int, start_symbol: int = SOS_TKN_IDX):
  src = src.to(DEVICE)
  src_mask = src_mask.to(DEVICE)

  memory = model.encode(src, src_mask) # context vector
  ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)
  for i in range(max_len - 1):
    memory = memory.to(DEVICE)
    tgt_mask = (generate_square_subsequent_mask(ys.size(0)).type(torch.bool)).to(DEVICE)

    out = model.decode(ys, memory, tgt_mask)
    out = out.transpose(0, 1)

    prob = model.output_layer(out[:, -1])
    _, next_word = torch.max(prob, dim=1)
    next_word = next_word.item()

    ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)

    if next_word == EOS_TKN_IDX:
      break

  return ys


def translate(model, src_sentence: str):
  model.eval()
  
  src = text_transform[SRC_LANGUAGE](src_sentence).view(-1, 1)
  num_tokens = src.shape[0]

  src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)
  tgt_tokens = greedy_decode(model, src, src_mask, max_len = num_tokens + 5).flatten()
  tgt_tokens = list(tgt_tokens.cpu().numpy())

  return " ".join(vocab_transform[TGT_LANGUAGE].lookup_tokens(tgt_tokens)).replace("<sos>", "").replace("<eos>", "")

print(translate(transformer, "Eine Gruppe von Menschen steht vor einem Iglu ."))

 A group of people stand in front of an igloo . 


In [ ]:
import random
val_iter = Multi30k(root='./data', split='valid', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))

for idx, sample in enumerate(val_iter):
  if idx >= 5: break
  src_sentence = sample[0]
  gt_sentence = sample[1]
  output_sentence = translate(transformer, src_sentence)

  print(f'dutch:         {src_sentence}')
  print(f'english(gt):   {gt_sentence}')
  print(f'english(pred): {output_sentence}')
  print('-' * 50)

dutch: Eine Gruppe von Männern lädt Baumwolle auf einen Lastwagen

english(gt): A group of men are loading cotton onto a truck

english(pred):  A group of men are loading into a truck of traffic . 
--------------------------------------------------
dutch: Ein Mann schläft in einem grünen Raum auf einem Sofa.

english(gt): A man sleeping in a green room on a couch.

english(pred):  A man is sleeping on a couch in a green room . 
--------------------------------------------------
dutch: Ein Junge mit Kopfhörern sitzt auf den Schultern einer Frau.

english(gt): A boy wearing headphones sits on a woman's shoulders.

english(pred):  A boy wearing headphones sits on his shoulders 's shoulders . 
--------------------------------------------------
dutch: Zwei Männer bauen eine blaue Eisfischerhütte auf einem zugefrorenen See auf

english(gt): Two men setting up a blue ice fishing hut on an iced over lake

english(pred):  Two men are setting up a blue piece of plastic on a lake . 
-------------